In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import OrderedDict

In [2]:
def openDataFile(name):
    entries = os.listdir('data/')
    i = 1
    for v in entries:
        print(str(i) + '.'+v)
        i = i + 1
    print('Seleccione el numero del archivo: ', name)
    print('En caso de no tener el archivo en data puede ingresar el directorio')
    aux = input()
    if len(aux) <= 5:
        i = 1
        for v in entries:
            if aux == str(i):
                return pd.read_excel('data/' + v)
            i = i + 1
    else: 
        return pd.read_excel(v)

In [3]:
def setMonthandValues(db):
    setActual = "setMonthandValues"
    lineaActual = 1
    month = []
    year = []
    value = []
    for index, row in db.iterrows():
        word = row['MES'].split()
        month.append(word[0])
        year.append(word[2])
        value.append(row['ACTUAL'] + row['ANTERIOR'])
    return month, year, value

In [4]:
def errorMessage(setActual, lineaActual, valor): 
    print('ERROR')
    print('Se ha detectado un valor inexistente en el Catalogo', setActual)
    print('Revisar en el documento de datos en la linea: ', lineaActual)
    print('El valor no registrado es: ', valor)
    print('Favor de agregar el valor faltante a el catalago y volver a intentar')
    print('Ingrese ENTER para continuar')
    print('Ingrese cualquier otra tecla para salir')
    aux = input()
    if not aux:
        return True
    else: 
        sys.exit()

In [5]:
def setGZ(gz, gzCat):
    global setActual
    global lineaActual
    global valor
    setActual = 'GZ'
    lineaActual = 1
    gz1 = []
    dr = []
    for value in gz:
        valor = value
        loc = gzCat.loc[gzCat.PLAZA == value]
        dr.append(loc['DR'].iloc[0])
        gz1.append(loc['GZ'].iloc[0])
        lineaActual = lineaActual + 1
    return dr, gz1

In [6]:
def setFab(fab):
    global setActual
    global lineaActual
    global valor
    setActual = 'Fabricante'
    lineaActual = 1
    fabr = []
    for value in fab:
        if value == "HMEX":
            fabr.append("HMEX")
        else:
            fabr.append("ABI")
    return fabr

In [7]:
def calculoPorGz(data):
    names = ["Gz", "Valor", "Mes"]
    df = pd.DataFrame(data, names)
    gp = df.groupby('Mes').sum().T
    
    return gp

In [17]:
def generateDatabase():
        
    ## Lectura de documentos y datos
    try: 
        ## Leer los archivos
        db = openDataFile('Input base: Scantrack (Suprmkt)')
            ## Quitar las plazas que no se ocupan
        db = db[db.PLAZA != "TOTAL AUTOS SCANNING MEXICO"]
        db = db[db.PLAZA != "CHIHUAHUA"]
        db = db[db.PLAZA != "JALAPA"]
        db = db[db.PLAZA != "REYNOSA"]
        ## Inicializar valores
        global setActual
        global lineaActual
        global value
        setActual = ''
        lineaActual = 10000
    except: 
        print('ERROR')
        print('Se ha generado un problema en la lectura del archivo')
        print('Favor de revisar estos documentos y volver a intentar')  
    
    flag = True
    while flag:
        os.system('cls')
        try:    ## Homologar los valores para simplificar la tabla             
            mo, yr, va  = setMonthandValues(db)
            month = pd.Series(mo)
            year = pd.Series(yr)
            value = pd.Series(va)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)
    
    flag = True
    while flag:
        os.system('cls')
        try:             
            f = setFab(db["COMPAÑÍA"])
            fab = pd.Series(f)
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)
    
    ## Obtener todos los valores con la llave de GZ         
    flag = True
    while flag: 
        gzCat = pd.read_excel("data/CatalagoScantrack.xlsx", sheet_name = 'Gz')
        os.system('cls')
        try:         
            g, d = setGZ(db['PLAZA'], gzCat)
            gz = pd.Series(g)
            dr = pd.Series(d)
            flag = False
        except:  
            flag = errorMessage(setActual, lineaActual, valor)
    
    gp = calculoPorGz([gz, value, db.MES])
    
    flag = True
    while flag:
        os.system('cls')
        try:              
            gp = calculoPorGz([gz, value, db.MES])
            flag = False
        except:
            flag = errorMessage(setActual, lineaActual, valor)            
            
    data = [db['PLAZA'], dr, gz, db["COMPAÑÍA"], db["MARCA"], db["SKU"], db["LIQUIDO"], db["SEGMENTO PRESENTACION"], db["DIAMANTE DE PRECIO CM"],db["ENVASE UNIF"] , db['PRESENTACION UNIF'],db["DIAMANTE DE PRECIO RM"], db["BWP"], db["Empaque"],value, month, year]
    name = ["PLAZA", "DR", "Gz", "COMPAÑÍA", "MARCA", "SKU", "LIQUIDO", "SEGMENTO PRESENTACION", "DIAMANTE DE PRECIO CM", "ENVASE UNIF", 'PRESENTACION UNIF', "DIAMANTE DE PRECIO RM", "BWP", "Empaque", "Valores", "Año", "Mes"]
    db = pd.DataFrame(data, name).T
    db.to_excel("Scantrack.xlsx", sheet_name = "Tabla_Para_Power_BI", index = False)
    


    
    return gp

        


In [19]:
generateDatabase()

1.Base Off Modern Marzo 2021.xlsx
2.Base Scantrack Marzo 2021.xlsx
3.CatalagoScantrack.xlsx
4.Catalogo711.xlsx
5.CatalogoADFy3B.xlsx
6.CatalogoCerveza.xlsx
7.CatalogoCidra.xlsx
8.CatalogoOxxo.xlsx
9.CatalogoSIX.xlsx
10.Check spaces.xlsx
11.Input711.xlsx
12.InputADFy3B.xlsx
13.InputCerveza.xlsx
14.InputCidra.xlsx
15.InputOxxo.xlsx
16.InputSIX.xlsx
17.SellOutRegion.xlsx
18.SellOutSix.xlsx
19.~$Base Scantrack Marzo 2021.xlsx
20.~$CatalagoScantrack.xlsx
21.~$Catalogo711.xlsx
22.~$CatalogoADFy3B.xlsx
23.~$CatalogoOxxo.xlsx
24.~$Input711.xlsx
25.~$InputCidra.xlsx
Seleccione el numero del archivo:  Input base: Scantrack (Suprmkt)
En caso de no tener el archivo en data puede ingresar el directorio
ERROR
Se ha generado un problema en la lectura del archivo
Favor de revisar estos documentos y volver a intentar
ERROR
Se ha detectado un valor inexistente en el Catalogo GZ
Revisar en el documento de datos en la linea:  257240
El valor no registrado es:  OCCIDENTE
Favor de agregar el valor faltante 

KeyboardInterrupt: Interrupted by user

1


## db.groupby('MES').sum().T

In [54]:
res = list(OrderedDict.fromkeys(db.MES))

In [20]:
        db = pd.read_excel("data/Base Scantrack Marzo 2021.xlsx")
        db = openDataFile('Input base: Scantrack (Suprmkt)')
            ## Quitar las plazas que no se ocupan
        db = db[db.PLAZA != "TOTAL AUTOS SCANNING MEXICO"]
        db = db[db.PLAZA != "CHIHUAHUA"]
        db = db[db.PLAZA != "JALAPA"]
        db = db[db.PLAZA != "REYNOSA"]
        mo, yr, va  = setMonthandValues(db)
        month = pd.Series(mo)
        year = pd.Series(yr)
        value = pd.Series(va)
        f = setFab(db["COMPAÑÍA"])
        fab = pd.Series(f)
        gzCat = pd.read_excel("data/CatalagoScantrack.xlsx", sheet_name = 'Gz')
        g, d = setGZ(db['PLAZA'], gzCat)
        gz = pd.Series(g)
        dr = pd.Series(d)

1.Base Off Modern Marzo 2021.xlsx
2.Base Scantrack Marzo 2021.xlsx
3.CatalagoScantrack.xlsx
4.Catalogo711.xlsx
5.CatalogoADFy3B.xlsx
6.CatalogoCerveza.xlsx
7.CatalogoCidra.xlsx
8.CatalogoOxxo.xlsx
9.CatalogoSIX.xlsx
10.Check spaces.xlsx
11.Input711.xlsx
12.InputADFy3B.xlsx
13.InputCerveza.xlsx
14.InputCidra.xlsx
15.InputOxxo.xlsx
16.InputSIX.xlsx
17.SellOutRegion.xlsx
18.SellOutSix.xlsx
19.~$Base Scantrack Marzo 2021.xlsx
20.~$CatalagoScantrack.xlsx
21.~$Catalogo711.xlsx
22.~$CatalogoADFy3B.xlsx
23.~$CatalogoOxxo.xlsx
24.~$Input711.xlsx
25.~$InputCidra.xlsx
Seleccione el numero del archivo:  Input base: Scantrack (Suprmkt)
En caso de no tener el archivo en data puede ingresar el directorio
2


In [21]:
db.MES

928       Enero de 2020
929       Enero de 2020
930       Enero de 2020
931       Enero de 2020
932       Enero de 2020
              ...      
281579    Marzo de 2021
281580    Marzo de 2021
281581    Marzo de 2021
281582    Marzo de 2021
281583    Marzo de 2021
Name: MES, Length: 257239, dtype: object

In [34]:
data = [gz, value, db.MES]
names = ["Gz", "Valor", "Mes"]
df = pd.DataFrame(data, names).T

In [36]:
gp = db.groupby('Mes').sum().T

KeyError: 'Mes'

In [35]:
df

,Gz,Valor,Mes
0,Noroeste,0.24,NaN
1,Noroeste,13.61,NaN
2,Noroeste,1.2,NaN
3,Noroeste,0.03,NaN
4,Noroeste,0.07,NaN
...,...,...,...
281579,NaN,NaN,Marzo de 2021
281580,NaN,NaN,Marzo de 2021
281581,NaN,NaN,Marzo de 2021
281582,NaN,NaN,Marzo de 2021


In [28]:
len(db.MES)

257239

In [38]:
db.MES[0]

KeyError: 0